In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/damoncrockett/ivpytest/src')
from ivpy import *
from ivpy.extract import extract,norm
from ivpy.utils import resize
import glob,os
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
/Users/damoncrockett/iv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/damoncrockett/iv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/damoncrockett/iv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/damoncrockett/iv/lib/python3.6/site-packages/t

In [2]:
f = '/Users/damoncrockett/Desktop/textureparams2.csv'
xf = pd.read_csv(f,header=None)

In [3]:
X = norm(xf.iloc[:,1:])

In [4]:
DIR = '/Users/damoncrockett/simspace/src/assets/json/img/'

In [5]:
# won't be used in the web app, but useful for futzing with the images prior to

xf['localpath'] = [DIR + item + '.jpeg' for item in xf[0]]

### Creating localhost paths

In [6]:
xf['localhostpath'] = ['http://localhost:8888/img/' + os.path.basename(item) for item in xf.localpath]

### Metadata

In [7]:
xf['edition'] = [item.split('_')[0] for item in xf[0]]

### Creation of simspaces

The target problem is visualizing the similarity spaces of different texture models. In order to visualize these spaces, we need dimension reduction (to 2 dimensions). For now, we'll just use t-SNE and UMAP, since they are different simspaces and can return 2-dimensional display coordinates.

In [8]:
from ivpy.reduce import pca,tsne,umap

In [9]:
xf[['xp','yp']] = norm(pca(X,n_components=2))
xf[['xt','yt']] = norm(tsne(X))
xf[['xu','yu']] = norm(umap(X))

### JSON creation

In [10]:
import json

In [11]:
def row2jso(xf,xcol,ycol,i):
    row = xf.loc[i]
    jso = {}
    jso['imgpath'] = row.localhostpath
    jso['x'] = row[xcol]
    jso['y'] = row[ycol]
    jso['edition'] = row.edition

    return jso

In [12]:
pcaArray = [row2jso(xf,'xp','yp',i) for i in xf.index]
tsneArray = [row2jso(xf,'xt','yt',i) for i in xf.index]
umapArray = [row2jso(xf,'xu','yu',i) for i in xf.index]

In [13]:
JSONDIR = '/Users/damoncrockett/simspace/src/assets/json/'

In [14]:
with open(JSONDIR+"sp_pca.json",'w') as outfile:
    json.dump(pcaArray,outfile)

with open(JSONDIR+"sp_tsne.json",'w') as outfile:
    json.dump(tsneArray,outfile)

with open(JSONDIR+"sp_umap.json",'w') as outfile:
    json.dump(umapArray,outfile)